In [39]:
import numpy as np 
import sys
import os
import re
def set_path():
    if sys.platform == 'darwin':
        print("Current system is macOS")
        main_fold_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'
    elif sys.platform == 'linux':
        print("Current system is Ubuntu")
        main_fold_path = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning'
    else:
        print("Current system is neither macOS nor Ubuntu")
    os.chdir(main_fold_path)
    
set_path()    


import pandas as pd 
def get_best_acc_epoch(csv_file):
    df = pd.read_csv(csv_file)
    index_of_best_acc = df['val_accuracy'].argmax()
    return index_of_best_acc

import glob 

def delete_files_starting_with(directory, prefix):
    pattern = os.path.join(directory, '*' + prefix + '*') 
    files_to_delete = glob.glob(pattern)
    for file in files_to_delete:
        try: 
            os.remove(file)
        except Exception as e:
            print(e)
        

model_name = 'cnn_transformer'
config_name = 'skf_v0mdd_classification_jamba'
model_config_dict = {
    'cnn_transformer': ['skf_v0mdd_classification_jamba'],
    'jamba': np.flip(['skf_v0mdd_classification_jamba', 
              'skf_v1_2layersmdd_classification_jamba', 
              'skf_v1_1layers_64_modelstatesmdd_classification_jamba',
              'skf_v2_sin_lrmdd_classification_jamba',
              'skf_v4_1e5_lrbeginmdd_classification_jamba',
              'skf_v5_no_mean_premdd_classification_jamba',
              'skf_v6_3layers_256_input_dimsmdd_classification_jamba',
              'skf_v7_3layers_256dims_9blocksmdd_classification_jamba',
              'skf_v8_2layersmdd_classification_jamba',
              'skf_v9_4layersmdd_classification_jamba',
              'skf_s0mdd_classification_jamba',
              'skf_s1_conv1d_longmdd_classification_jamba',
              'skf_s2_conv1d_1000epochs_1e9lrmdd_classification_jamba',
              'skf_s3_1000epochs_1e6lrmdd_classification_jamba',
              'skf_s4_1e-5lrmdd_classification_jamba',
              'skf_s5mdd_classification_jamba',
              'skf_s6_replicationmdd_classification_jamba',
              'skf_s7_replicat_5000warmupmdd_classification_jamba']),
    'mamba': ['skf_v0mdd_classification_mamba']
}
validation_name = 'SKF_holdout/stratified_nested_10_CV_fold'
num_of_k_fold = 10

# reading the last index of result
def find_path_acc(path):
    with open(path, 'r') as f:
        acc = f.read()
    acc = acc.split('\n')[-2]
    acc = re.findall(r'accuracy: (\d+\.\d+)', acc)[0]
    
    return np.float(acc)

def read_model_config_result(model_name, config_name, validation_name='SKF_holdout/stratified_nested_10_CV_fold', num_of_k_fold=10):
    test_acc = []
    val_acc = []
    index_best = []
    for k in range(num_of_k_fold):
        path = f'results/{model_name}/diagnosis514/{config_name}/{validation_name}-{k}' 
        fold_test_acc = find_path_acc(path + '/test_acc.txt')
        fold_val_acc = find_path_acc(path + '/val_acc.txt')
        fold_index_best = get_best_acc_epoch(path + '/history.csv')
        index_best.append(fold_index_best)
        test_acc.append(fold_test_acc)
        val_acc.append(fold_val_acc)
        
        # delete checkpoint files, in case the space is not enough
        # delete_files_starting_with(path, 'checkpoint')
        
    print('model_name:', model_name, config_name)
    print('test_acc:', np.mean(test_acc))
    print('val_acc:', np.mean(val_acc))
    print('index_best:', np.mean(index_best))

for model_name, config_names in model_config_dict.items():
    for config_name in config_names:
        read_model_config_result(model_name, config_name, validation_name, num_of_k_fold)

Current system is Ubuntu
model_name: cnn_transformer skf_v0mdd_classification_jamba
test_acc: 0.5333330000000001
val_acc: 0.6577759999999999
index_best: 78.5
model_name: jamba skf_s7_replicat_5000warmupmdd_classification_jamba
test_acc: 0.5705889999999999
val_acc: 0.75111
index_best: 130.2
model_name: jamba skf_s6_replicationmdd_classification_jamba
test_acc: 0.6882349999999999
val_acc: 0.72
index_best: 156.4
model_name: jamba skf_s5mdd_classification_jamba
test_acc: 0.605883
val_acc: 0.7422229999999999
index_best: 110.2
model_name: jamba skf_s4_1e-5lrmdd_classification_jamba
test_acc: 0.6215680000000001
val_acc: 0.733333
index_best: 88.4
model_name: jamba skf_s3_1000epochs_1e6lrmdd_classification_jamba
test_acc: 0.684315
val_acc: 0.7044439999999998
index_best: 78.9
model_name: jamba skf_s2_conv1d_1000epochs_1e9lrmdd_classification_jamba
test_acc: 0.5333330000000001
val_acc: 0.7666670000000001
index_best: 196.8
model_name: jamba skf_s1_conv1d_longmdd_classification_jamba
test_acc: 0.68

/tmp/ipykernel_4075133/2432710358.py:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(acc)


In [15]:
# read the best acc and its corresponding epoch 
import pandas as pd 
def get_best_acc_epoch(csv_file):
    df = pd.read_csv(csv_file)
    index_of_best_acc = df['val_accuracy'].argmax()
    return index_of_best_acc
# csv_file = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/results/jamba/diagnosis514/skf_s2_conv1d_1000epochs_1e9lrmdd_classification_jamba/SKF_holdout/stratified_nested_10_CV_fold-9/history.csv'
# df = pd.read_csv(csv_file)
# index_of_best_acc = df['val_accuracy'].argmax()
# index_of_best_acc